<a href="https://colab.research.google.com/github/PSivaMallikarjun/AI-Driven-Agrochemical-Testing/blob/main/AI_Driven_Agrochemical_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AI-Driven HPLC and GLPC Testing for Agrochemicals

# Overview

This project leverages AI to enhance High-Performance Liquid Chromatography (HPLC) and Gas-Liquid Chromatography (GLPC) testing in agrochemicals. It provides an interactive interface for users to analyze, compare, and optimize chemical formulations for improved efficiency and compliance.

# Features

* Chemical Selection: Dropdown list of commonly used agrochemicals with the ability to add/remove entries.

* Formulation Input: Users can enter the percentage composition of the formulation.

* HPLC/GLPC Data Analysis: Compare actual vs. desired results with visualization.

* Undo Feature: Allows reverting the last action for better usability.

* CRUD Operations: Add, update, and delete chemicals in the system dynamically.

In [1]:
!pip install gradio pandas numpy matplotlib seaborn plotly scikit-learn tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00


# Installation

* To set up the project locally, install the required dependencies:

# Usage

* Run the application using:

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import json
from sklearn.linear_model import LinearRegression

# JSON file for storing chemicals
CHEMICALS_FILE = "chemicals.json"

# Load or initialize chemical sources
def load_chemicals():
    try:
        with open(CHEMICALS_FILE, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return ["EC", "SC", "Paraquat Grass Killer", "Granules", "Pest Control"]

# Save chemicals to JSON file
def save_chemicals(chemicals):
    with open(CHEMICALS_FILE, "w") as file:
        json.dump(chemicals, file)

# Global chemical list & action history for undo
chemical_sources = load_chemicals()
action_history = []

# Function to add a new chemical
def add_chemical(new_chemical):
    global chemical_sources
    if new_chemical and new_chemical not in chemical_sources:
        chemical_sources.append(new_chemical)
        action_history.append(("add", new_chemical))
        save_chemicals(chemical_sources)
    return chemical_sources

# Function to delete a chemical
def delete_chemical(chemical):
    global chemical_sources
    if chemical in chemical_sources:
        chemical_sources.remove(chemical)
        action_history.append(("delete", chemical))
        save_chemicals(chemical_sources)
    return chemical_sources

# Undo last action
def undo_action():
    if action_history:
        last_action, chemical = action_history.pop()
        if last_action == "add":
            chemical_sources.remove(chemical)
        elif last_action == "delete":
            chemical_sources.append(chemical)
        save_chemicals(chemical_sources)
    return chemical_sources

# AI Model Placeholder for Efficiency Analysis
def ai_efficiency_prediction(actual, desired):
    model = LinearRegression()
    x = np.array(actual).reshape(-1, 1)
    y = np.array(desired)
    model.fit(x, y)
    efficiency_score = model.predict(x)[-1]
    improvement_points = np.round((desired[-1] - efficiency_score) * 100 / desired[-1], 2)
    return efficiency_score, improvement_points

# Function to Generate Candle & Line Graphs
def generate_graphs(actual, desired, chemical):
    fig, ax = plt.subplots(figsize=(8,5))

    # Line Graph
    plt.plot(actual, label="Actual HPLC/GLPC", marker="o", linestyle="-", color="blue")
    plt.plot(desired, label="Desired HPLC/GLPC", marker="s", linestyle="--", color="red")
    plt.xlabel("Sample Tests")
    plt.ylabel("Chemical Efficiency (%)")
    plt.title(f"{chemical} - HPLC & GLPC Efficiency")
    plt.legend()
    plt.grid(True)

    # Save and display plot
    plt.savefig("line_graph.png")

    # Candle Graph using Plotly
    fig_candle = go.Figure(data=[go.Candlestick(
        x=list(range(len(actual))),
        open=actual, high=[max(actual)], low=[min(actual)], close=desired
    )])

    fig_candle.update_layout(title=f"{chemical} - Efficiency Variation", xaxis_title="Sample Tests",
                             yaxis_title="Efficiency (%)", template="plotly_dark")

    return "line_graph.png", fig_candle

# Gradio UI for Agrochemical Testing
def agro_test(chemical, formulation, actual_values, desired_values):
    actual = list(map(float, actual_values.split(",")))
    desired = list(map(float, desired_values.split(",")))

    efficiency, improvement = ai_efficiency_prediction(actual, desired)
    line_graph, candle_graph = generate_graphs(actual, desired, chemical)

    report = f"**Efficiency Score:** {efficiency:.2f}%\n\n"
    report += f"**Improvement Needed:** {improvement:.2f}%\n\n"
    report += f"Suggested Adjustments: Increase purity levels, optimize formulation ratios.\n"

    return report, line_graph, candle_graph

# Gradio UI Components
with gr.Blocks() as app:
    gr.Markdown("# AI-Driven Agrochemical Testing 🌱")

    # CRUD Operations Section
    with gr.Row():
        chem_dropdown = gr.Dropdown(choices=chemical_sources, label="Select Chemical", interactive=True)
        add_input = gr.Textbox(label="Add New Chemical")
        add_button = gr.Button("➕ Add")
        delete_button = gr.Button("❌ Delete Selected")
        undo_button = gr.Button("↩️ Undo")

    add_button.click(add_chemical, inputs=[add_input], outputs=[chem_dropdown])
    delete_button.click(delete_chemical, inputs=[chem_dropdown], outputs=[chem_dropdown])
    undo_button.click(undo_action, outputs=[chem_dropdown])

    # Agro Test Section
    formulation_input = gr.Textbox(label="Enter Formulation %")
    actual_input = gr.Textbox(label="Actual HPLC/GLPC Results (comma-separated)")
    desired_input = gr.Textbox(label="Desired HPLC/GLPC Results (comma-separated)")
    analyze_button = gr.Button("🔍 Analyze")

    report_output = gr.Textbox(label="Efficiency Report")
    line_graph_output = gr.Image(label="Line Graph")
    candle_graph_output = gr.Plot(label="Candle Graph")

    analyze_button.click(
        agro_test,
        inputs=[chem_dropdown, formulation_input, actual_input, desired_input],
        outputs=[report_output, line_graph_output, candle_graph_output]
    )

# Launch the Gradio app
app.launch(debug=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b1e9213085d0fdd23c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:227: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ['EC', 'SC', 'Paraquat Grass Killer', 'Granules', 'Pest Control'] or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:227: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ['EC', 'SC', 'Paraquat Grass Killer', 'Granules', 'Pest Control', 'Lambda-Cyhalothrin 5% EC'] or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:227: UserWarning:

The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: ['EC', 'SC', 'Paraquat Grass Killer', 'Granules', 'Pest Control', 'Lambda-Cyhalothrin 5% EC'] or set allow_custom_value=True.

